<a href="https://colab.research.google.com/github/yoondonggyu/KTB-Project/blob/main/%EA%B3%BC%EC%A0%9C_1%EB%B2%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. 자신이 생각하는 가장 깔끔한 트랜스포머 모델을 파이토치로 구현하고 리얼한 데이터셋으로 학습과 추론을 해보세요.

2. 1번의 코드를 바탕으로 트랜스포머의 얼개를 설명하는 보고서를 작성해보세요.


## PyTorch 내장(nn.Transformer)

* 모델 : PyTorch의 torch.nn.Transformer 클래스를 사용하여 트랜스포머 구조(인코더-디코더)를 단 한 줄로 생성합니다.
* 장점
   * 코드의 길이가 극도로 짧고 깔끔합니다.
   * 실제 연구나 개발에서 트랜스포머 레이어를 빠르게 활용할 때 가장 많이 쓰는 방법
   * 파라미터 설정만으로 인코더/디코더 개수를 조절할 수 있음

* 단점
  * 트랜스포머의 내부 동작(Self-Attention , LayerNorm 등)을 직접 구현해보는 경험은 할 수 없음

  

In [ ]:
import torch
import torch.nn as nn
import math

class TransformerModel(nn.Module):
    def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout=0.5):
        super().__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(d_model, dropout)

        # 🌟 핵심: nn.TransformerEncoderLayer와 nn.TransformerEncoder를 사용
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)

        # 입력(토큰)을 d_model 크기의 벡터로 변환하는 임베딩 레이어
        self.encoder = nn.Embedding(ntoken, d_model)
        self.d_model = d_model

        # 최종 출력을 ntoken (단어/토큰 개수)으로 변환하는 선형 레이어 (출력 레이어)
        self.decoder = nn.Linear(d_model, ntoken)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask):
        # 1. 임베딩: 입력 시퀀스를 벡터로 변환
        src = self.encoder(src) * math.sqrt(self.d_model)

        # 2. 포지셔널 인코딩: 시퀀스 순서 정보 추가
        src = self.pos_encoder(src)

        # 3. 트랜스포머 인코더: 핵심 연산 수행
        output = self.transformer_encoder(src, src_mask)

        # 4. 출력 레이어: 최종 결과를 토큰 확률 분포로 변환
        output = self.decoder(output)
        return output

# 필수 구성 요소: 포지셔널 인코딩 (Positional Encoding) 정의
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        x: [시퀀스 길이, 배치 크기, 임베딩 차원] 형태
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
# 하이퍼파라미터 설정
BOS_TOKEN = 1  # Start Of Sentence 토큰
EOS_TOKEN = 2  # End Of Sentence 토큰
PAD_TOKEN = 0  # Padding 토큰
NTokens = 10   # 전체 토큰(단어) 개수 (0~9)
EmbdSize = 256 # 임베딩 차원
NHead = 4      # 멀티 헤드 어텐션 헤드 개수
NHid = 256     # 피드포워드 네트워크의 은닉 차원
NLayers = 2    # 트랜스포머 레이어 개수
SeqLen = 5     # 시퀀스 길이
BatchSize = 32

# 가상 데이터 생성 함수
def generate_batch(batch_size, seq_len, n_tokens):
    # [시퀀스 길이, 배치 크기] 형태로 생성 (PyTorch Transformer는 이 형태를 선호)
    data = torch.randint(3, n_tokens, (seq_len, batch_size)) # 3~9 사이의 토큰

    # 🌟 Copy Task: 입력(src)과 목표(tgt)가 동일함
    src = data
    tgt = data

    return src, tgt

# 마스크 생성 함수 (패딩과 미래 토큰 방지)
def generate_square_subsequent_mask(size):
    # 룩-어헤드(Look-Ahead) 마스크: 디코더에서 미래 토큰을 보지 못하게 함
    mask = (torch.triu(torch.ones(size, size)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [ ]:
# 1. 모델 초기화
model = TransformerModel(NTokens, EmbdSize, NHead, NHid, NLayers)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN) # PAD_TOKEN은 손실 계산에서 무시
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 마스크 준비
src_mask = generate_square_subsequent_mask(SeqLen)

# 2. 간단한 학습 루프 (100 에폭)
print("--- 학습 시작 ---")
model.train()
for epoch in range(100):
    src, tgt = generate_batch(BatchSize, SeqLen, NTokens)

    # 🌟 PyTorch Transformer Encoder만 사용하므로, 입력(src)과 목표(tgt)가 같고
    #    디코더 마스크 대신 인코더 마스크만 사용하거나 (Copy Task이므로) 마스크를 무시할 수 있음.
    #    Seq2Seq를 구현하려면 nn.Transformer() 전체를 사용하고 tgt_mask가 필요합니다.
    #    여기서는 Copy Task & nn.TransformerEncoder 예시이므로, src를 입력받아 다음 토큰을 예측하는 방식으로 간주합니다.

    # nn.TransformerEncoder는 Self-Attention만 수행하므로, src_mask는 패딩 마스크 역할만 하거나 (여기서는 패딩 없음),
    # 일반적인 언어 모델의 룩어헤드 마스크 역할을 할 수 있습니다.
    # Copy Task이므로 룩어헤드 마스크를 사용합니다.
    output = model(src, src_mask)

    # [시퀀스 길이, 배치 크기, NTokens] -> [배치 크기 * 시퀀스 길이, NTokens]
    loss = criterion(output.reshape(-1, NTokens), tgt.reshape(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:3d} | Loss: {loss.item():.4f}")

print("--- 학습 완료 ---")

# 3. 간단한 추론 (Evaluation)
model.eval()
print("\n--- 추론 시작 (Copy Task) ---")
test_src, _ = generate_batch(1, SeqLen, NTokens) # 배치 크기 1
print(f"입력 시퀀스 (Source): \n{test_src.squeeze().tolist()}")

with torch.no_grad():
    output = model(test_src, src_mask[:test_src.size(0), :test_src.size(0)])

    # 가장 높은 확률을 가진 토큰을 선택하여 예측 시퀀스 생성
    predicted_sequence = output.argmax(dim=-1).squeeze().tolist()

print(f"예측 시퀀스 (Predicted): \n{predicted_sequence}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:392: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


--- 학습 시작 ---
Epoch  20 | Loss: 0.0084
Epoch  40 | Loss: 0.0018
Epoch  60 | Loss: 0.0015
Epoch  80 | Loss: 0.0011
Epoch 100 | Loss: 0.0007
--- 학습 완료 ---

--- 추론 시작 (Copy Task) ---
입력 시퀀스 (Source): 
[4, 6, 3, 8, 7]
예측 시퀀스 (Predicted): 
[4, 6, 3, 8, 7]


In [ ]:
import torch
# ... (이전 코드의 모든 import와 클래스 정의 유지)
# ... (TransformerModel, PositionalEncoding 클래스 정의 유지)

# 🌟 1. 정확도 계산 함수 추가 🌟
def calculate_accuracy(output, target, pad_idx):
    """
    모델의 출력(output)과 실제 목표(target)를 비교하여 정확도를 계산합니다.
    패딩 토큰(pad_idx)은 계산에서 제외합니다.
    """
    # output: [시퀀스 길이, 배치 크기, NTokens]
    # target: [시퀀스 길이, 배치 크기]

    # 예측된 클래스 인덱스 (가장 높은 확률을 가진 토큰)
    predicted_tokens = output.argmax(dim=-1) # [시퀀스 길이, 배치 크기]

    # 패딩 토큰이 아닌 위치에 대해서만 마스크 생성
    mask = (target != pad_idx)

    # 예측이 정확한 위치 확인
    correct_predictions = (predicted_tokens == target)

    # 마스크를 적용하여 패딩을 제외한 정확한 예측 수 카운트
    correct_sum = (correct_predictions & mask).sum().item()

    # 패딩을 제외한 전체 토큰 수 카운트
    total_tokens = mask.sum().item()

    # 정확도 계산
    accuracy = correct_sum / total_tokens if total_tokens > 0 else 0.0

    return accuracy

In [ ]:
# 하이퍼파라미터 설정 (이전과 동일)
PAD_TOKEN = 0  # Padding 토큰 (손실 및 정확도 계산에서 무시할 토큰)
NTokens = 10
EmbdSize = 256
NHead = 4
NHid = 256
NLayers = 2
SeqLen = 5
BatchSize = 32

# 모델 초기화 (이전과 동일)
# model, criterion, optimizer, src_mask 정의 (이전 코드와 동일하게)
model = TransformerModel(NTokens, EmbdSize, NHead, NHid, NLayers)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN) # PAD_TOKEN은 손실 계산에서 무시
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
src_mask = generate_square_subsequent_mask(SeqLen)

# 🌟 3. 수정된 학습 루프 🌟
print("--- 모델 학습 시작 (손실 및 정확도 출력) ---")
model.train()
for epoch in range(100):
    src, tgt = generate_batch(BatchSize, SeqLen, NTokens)

    # 마스크 크기가 배치마다 바뀔 경우를 대비해 마스크 크기 조정 (Copy Task에서는 항상 고정)
    current_src_mask = src_mask[:src.size(0), :src.size(0)]

    output = model(src, current_src_mask)

    # 손실 계산
    # output: [SeqLen, BatchSize, NTokens] -> [BatchSize * SeqLen, NTokens]
    # tgt: [SeqLen, BatchSize] -> [BatchSize * SeqLen]
    loss = criterion(output.reshape(-1, NTokens), tgt.reshape(-1))

    # 🌟 정확도 계산
    accuracy = calculate_accuracy(output, tgt, PAD_TOKEN)

    # 역전파 및 최적화
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 20 == 0:
        print(f"Epoch {epoch+1:3d} | Loss: {loss.item():.4f} | Accuracy: {accuracy:.4f}")

print("--- 학습 완료 ---")

# 4. 간단한 추론 (Evaluation) (이전 코드와 동일)
model.eval()
print("\n--- 추론 시작 (Copy Task) ---")
test_src, _ = generate_batch(1, SeqLen, NTokens) # 배치 크기 1
print(f"입력 시퀀스 (Source): \n{test_src.squeeze().tolist()}")

with torch.no_grad():
    output = model(test_src, src_mask[:test_src.size(0), :test_src.size(0)])
    predicted_sequence = output.argmax(dim=-1).squeeze().tolist()

print(f"예측 시퀀스 (Predicted): \n{predicted_sequence}")

--- 모델 학습 시작 (손실 및 정확도 출력) ---
Epoch  20 | Loss: 0.0056 | Accuracy: 1.0000
Epoch  40 | Loss: 0.0020 | Accuracy: 1.0000
Epoch  60 | Loss: 0.0010 | Accuracy: 1.0000
Epoch  80 | Loss: 0.0011 | Accuracy: 1.0000
Epoch 100 | Loss: 0.0008 | Accuracy: 1.0000
--- 학습 완료 ---

--- 추론 시작 (Copy Task) ---
입력 시퀀스 (Source): 
[5, 5, 8, 6, 9]
예측 시퀀스 (Predicted): 
[5, 5, 8, 6, 9]


## 허깅페이스 transformers 라이브러리 활용(가장 실용적이고 강력함)

* 모델 : 허깅페이스(Hugging Face) transformers 라이브러리의 사전 학습된 모델(예:BERT, GPT-2) 또는 그 기반 구조를 사용합니다.
* 장점
   * 실제 산업에서 가장 많이 사용되는 방법으로, 바로 실무에 적용 가능한 결과물을 얻을 수 있습니다.  
   * 사전 학습된 모델을 활용하여 적은 데이터로도 높은 성능을 얻을 수 있습니다. (파인튜닝)

* 단점
  * 순수한 트랜스포머 논문의 아키텍처 구현과는 거리가 있으며, 라이브러리 종속성이 높습니다.
  
  

In [ ]:
!pip install transformers datasets accelerate -U
!pip install accelerate>=0.21.0

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

# 1. 가상 데이터셋 생성 (긍정/부정 텍스트 분류)
data = {
    'text': [
        "이 영화 정말 재미있었어요!",
        "너무 지루하고 실망스러웠습니다.",
        "최고의 연기, 다시 보고 싶네요.",
        "시간 낭비였고 추천하고 싶지 않아요.",
        "오랜만에 수작을 만났네요.",
        "기대 이하였고 내용이 부실합니다."
    ],
    'label': [1, 0, 1, 0, 1, 0] # 1: 긍정, 0: 부정
}

# Hugging Face Dataset 객체로 변환
hf_dataset = Dataset.from_dict(data)

# 데이터셋을 학습(train)과 검증(eval) 세트로 분리
train_test_split = hf_dataset.train_test_split(test_size=0.3)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# 2. 토크나이저 로드
# 한국어 BERT 모델인 'monologg/kobert'를 사용합니다.
MODEL_NAME = "monologg/kobert"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# 3. 데이터 토크나이징 함수 정의
def tokenize_function(examples):
    # max_length=128로 시퀀스 길이를 제한하고, 패딩과 잘라내기를 수행합니다.
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# 4. 데이터셋에 토크나이징 적용
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

# PyTorch 학습에 필요 없는 'text' 컬럼은 제거하고, 'input_ids', 'attention_mask', 'label'만 남깁니다.
tokenized_train_dataset = tokenized_train_dataset.remove_columns(["text"])
tokenized_eval_dataset = tokenized_eval_dataset.remove_columns(["text"])

The repository monologg/kobert contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/monologg/kobert .
 You can inspect the repository content at https://hf.co/monologg/kobert.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# 1. 사전 학습된 모델 로드 (시퀀스 분류를 위한 모델)
# num_labels=2는 긍정/부정 두 개의 클래스를 의미합니다.
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# 2. 평가 지표 함수 정의
def compute_metrics(p):
    # 예측된 로짓(logits)에서 argmax를 취하여 예측 레이블을 구합니다.
    preds = np.argmax(p.predictions, axis=1)

    # 정확도(Accuracy) 계산
    accuracy = accuracy_score(p.label_ids, preds)

    # 정밀도, 재현율, F1-score 계산
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='binary')

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }

# 3. TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',          # 결과물 저장 경로
    num_train_epochs=5,              # 총 학습 에폭 수
    per_device_train_batch_size=8,   # 디바이스당 학습 배치 크기
    per_device_eval_batch_size=8,    # 디바이스당 평가 배치 크기
    warmup_steps=500,                # 웜업 스텝 수 (학습률 조정)
    weight_decay=0.01,               # 가중치 감소 (규제화)
    logging_dir='./logs',            # 로그 저장 디렉토리
    logging_steps=100,
    # evaluation_strategy="epoch",     # 매 에폭마다 평가 수행
    # save_strategy="epoch",           # 매 에poch마다 체크포인트 저장
    # load_best_model_at_end=True,     # 학습 종료 시 최적 모델 로드
    # metric_for_best_model="accuracy" # 최적 모델 선택 기준


)

# 4. Trainer 초기화
trainer = Trainer(
    model=model,                         # 훈련할 모델
    args=training_args,                  # 훈련 인자
    train_dataset=tokenized_train_dataset, # 학습 데이터셋
    eval_dataset=tokenized_eval_dataset,   # 평가 데이터셋
    compute_metrics=compute_metrics,     # 평가 지표 함수
)

# 5. 모델 학습 시작 (파인튜닝)
print("--- 모델 파인튜닝 시작 ---")
trainer.train()
print("--- 모델 파인튜닝 완료 ---")

# 6. 최종 평가
print("\n--- 최종 평가 결과 ---")
eval_results = trainer.evaluate()
print(eval_results)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


--- 모델 파인튜닝 시작 ---


Step,Training Loss


--- 모델 파인튜닝 완료 ---

--- 최종 평가 결과 ---


{'eval_loss': 0.6737902164459229, 'eval_accuracy': 1.0, 'eval_precision': 1.0, 'eval_recall': 1.0, 'eval_f1': 1.0, 'eval_runtime': 0.0168, 'eval_samples_per_second': 119.167, 'eval_steps_per_second': 59.583, 'epoch': 5.0}


## Attention Is All You Need 논문 기반 핵심 모듈 직접 구현

* 모델 : 논문(Attention is all you need) 에서 제시한 핵슴 모듈(Multi-Head Attention, Feed-Forward Network, Positional Encoding 등) 만을 직접 PyTorch로 구현합니다.
* 장점
   * 트랜스포머의 내부 구조와 동작 원리를 완벽하게 파악할 수 있습니다.
   * 모든 구성 요소를 직접 제어할 수 있어 커스터마이징이 용이합니다.

* 단점
  * 코드가 길고, 디버깅이 필요하며, 학습 시간이 가장 오래 걸릴 수 있습니다.

```
# 코드로 형식 지정됨
```


  

In [ ]:
import torch
import torch.nn as nn
import math

class PositionalEncoding(nn.Module):
    """
    논문에서 제시된 사인 및 코사인 함수를 이용한 위치 인코딩 구현
    """
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        # pe: [max_len, 1, d_model] 크기의 위치 인코딩 행렬
        pe = torch.zeros(max_len, d_model)

        # position: [max_len, 1]
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

        # div_term: 10000^(-2i/d_model). i가 0, 2, 4...일 때의 값.
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))

        # 사인(짝수 인덱스) 및 코사인(홀수 인덱스) 적용
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        # [max_len, d_model] -> [max_len, 1, d_model] (배치 차원 추가)
        pe = pe.unsqueeze(0).transpose(0, 1)

        # 모델의 파라미터로 학습되지 않도록 등록
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: 입력 임베딩, [시퀀스 길이, 배치 크기, d_model]
        """
        # 입력 임베딩에 위치 인코딩 값 더하기
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    """
    논문의 Scaled Dot-Product Attention과 Multi-Head 메커니즘 구현
    """
    def __init__(self, d_model, n_head, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.d_k = d_model // n_head # 각 헤드의 차원

        # Query, Key, Value를 위한 선형 레이어
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        # 최종 출력을 위한 선형 레이어
        self.fc = nn.Linear(d_model, d_model)

        self.dropout = nn.Dropout(dropout)
        self.scale = torch.sqrt(torch.FloatTensor([self.d_k])) # 스케일링 상수

    def forward(self, query, key, value, mask=None):

        # 1. Q, K, V 변환 및 헤드 분할
        # Q, K, V: [배치 크기, 시퀀스 길이, d_model]

        # -> [배치 크기, 시퀀스 길이, n_head, d_k] -> [배치 크기, n_head, 시퀀스 길이, d_k]
        Q = self.w_q(query).view(query.shape[0], -1, self.n_head, self.d_k).transpose(1, 2)
        K = self.w_k(key).view(key.shape[0], -1, self.n_head, self.d_k).transpose(1, 2)
        V = self.w_v(value).view(value.shape[0], -1, self.n_head, self.d_k).transpose(1, 2)

        # 2. Scaled Dot-Product Attention 계산
        # energy = Q @ K^T / sqrt(d_k)
        # energy: [배치 크기, n_head, 시퀀스 길이, 시퀀스 길이]
        energy = torch.matmul(Q, K.transpose(-2, -1)) / self.scale.to(Q.device)

        # 3. 마스킹 적용 (선택 사항)
        if mask is not None:
            # 마스크가 적용되는 위치는 -1e9 (매우 작은 값)로 설정
            energy = energy.masked_fill(mask == 0, -1e9)

        # 4. Softmax 적용하여 Attention Weights (가중치) 계산
        # attention: [배치 크기, n_head, 시퀀스 길이, 시퀀스 길이]
        attention = F.softmax(energy, dim=-1)

        # 5. Attention Weights와 Value 행렬 곱
        # x: [배치 크기, n_head, 시퀀스 길이, d_k]
        x = torch.matmul(self.dropout(attention), V)

        # 6. 헤드 결합 (Concatenate Heads)
        # x: [배치 크기, 시퀀스 길이, n_head * d_k (d_model)]
        x = x.transpose(1, 2).contiguous().view(query.shape[0], -1, self.d_model)

        # 7. 최종 선형 변환
        x = self.fc(x)

        return x, attention

In [ ]:
class TransformerEncoderLayer(nn.Module):
    """
    트랜스포머 인코더의 단일 레이어 구성
    """
    def __init__(self, d_model, n_head, d_hid, dropout=0.1):
        super().__init__()

        # Self-Attention 서브 레이어
        self.self_attn = MultiHeadAttention(d_model, n_head, dropout)

        # Feed-Forward 서브 레이어 (간소화)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_hid),
            nn.ReLU(),
            nn.Linear(d_hid, d_model),
            nn.Dropout(dropout)
        )

        # Layer Normalization 및 드롭아웃
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src, src_mask=None):

        # 1. Self-Attention 서브 레이어
        # Residual connection + Layer Normalization
        _src, _ = self.self_attn(src, src, src, src_mask)
        src = self.norm1(src + self.dropout(_src))

        # 2. Feed-Forward 서브 레이어
        # Residual connection + Layer Normalization
        _src = self.feed_forward(src)
        src = self.norm2(src + self.dropout(_src))

        return src

In [ ]:
import torch

# 모델 하이퍼파라미터 설정 (구현하신 클래스의 __init__과 일치해야 함)
d_model = 512    # 임베딩 차원
n_head = 8       # 멀티 헤드 개수
d_hid = 2048     # Feed-Forward 네트워크 은닉 차원
dropout = 0.1
seq_len = 10     # 시퀀스 길이
batch_size = 2   # 배치 크기

# 테스트를 위한 가상 입력 임베딩 생성
# [시퀀스 길이, 배치 크기, d_model] 형태로 가정 (Transformer 구현 표준)
dummy_input = torch.randn(seq_len, batch_size, d_model)

print(f"가상 입력 크기: {dummy_input.shape}")

가상 입력 크기: torch.Size([10, 2, 512])


In [ ]:
# PositionalEncoding 클래스 인스턴스 생성
pe_module = PositionalEncoding(d_model, dropout)

# PE 모듈 실행
pe_output = pe_module(dummy_input)

print("\n--- PositionalEncoding 테스트 ---")
print(f"입력 크기: {dummy_input.shape}")
print(f"출력 크기: {pe_output.shape}")
# PE가 잘 적용됐는지 확인 (입력과 출력이 달라야 함)
print(f"PE 적용 후 입력과 출력의 차이 (L1 Norm): {torch.linalg.norm(dummy_input - pe_output):.4f}")


--- PositionalEncoding 테스트 ---
입력 크기: torch.Size([10, 2, 512])
출력 크기: torch.Size([10, 2, 512])
PE 적용 후 입력과 출력의 차이 (L1 Norm): 82.4915


In [ ]:
# MultiHeadAttention 클래스 인스턴스 생성
mha_module = MultiHeadAttention(d_model, n_head, dropout)

# MHA 모듈 실행 (Query, Key, Value 모두 dummy_input 사용)
# [배치 크기, 시퀀스 길이, d_model] 형태로 가정
# 실제 구현하신 MHA의 입력 형태가 [배치 크기, 시퀀스 길이, d_model]로 가정합니다.
# (만약 [시퀀스 길이, 배치 크기, d_model]이었다면 .transpose(0, 1)이 필요함)
dummy_input_mha = dummy_input.transpose(0, 1) # [배치 크기, 시퀀스 길이, d_model]로 변환

mha_output, attention_weights = mha_module(
    dummy_input_mha, dummy_input_mha, dummy_input_mha
)

print("\n--- MultiHeadAttention 테스트 ---")
print(f"입력 크기: {dummy_input_mha.shape}")
print(f"출력 크기: {mha_output.shape}")
# 어텐션 가중치의 크기: [배치 크기, n_head, 시퀀스 길이, 시퀀스 길이]
print(f"어텐션 가중치 크기: {attention_weights.shape}")


--- MultiHeadAttention 테스트 ---
입력 크기: torch.Size([2, 10, 512])
출력 크기: torch.Size([2, 10, 512])
어텐션 가중치 크기: torch.Size([2, 8, 10, 10])


In [ ]:
# TransformerEncoderLayer 클래스 인스턴스 생성
encoder_layer = TransformerEncoderLayer(d_model, n_head, d_hid, dropout)

# 인코더 레이어 실행
# 입력 형태: [배치 크기, 시퀀스 길이, d_model]
encoder_output = encoder_layer(dummy_input_mha)

print("\n--- TransformerEncoderLayer 테스트 ---")
print(f"입력 크기: {dummy_input_mha.shape}")
print(f"출력 크기: {encoder_output.shape}")


--- TransformerEncoderLayer 테스트 ---
입력 크기: torch.Size([2, 10, 512])
출력 크기: torch.Size([2, 10, 512])


In [ ]:
import torch.nn as nn

class FullTransformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, n_layers, n_head, d_hid, max_len, dropout):
        super().__init__()

        # 🌟 구현 필수 요소 🌟
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(trg_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len) # 이전에 구현하신 PE 사용

        # 🌟 직접 구현한 Encoder Layer를 여러 층으로 쌓습니다.
        encoder_layer = TransformerEncoderLayer(d_model, n_head, d_hid, dropout)
        self.encoder = nn.TransformerEncoder(encoder_layer, n_layers)

        # 🌟 디코더 구조는 인코더와 유사하게 구성해야 합니다.
        # ... (Cross-Attention이 포함된 TransformerDecoderLayer가 필요)

        self.fc_out = nn.Linear(d_model, trg_vocab_size) # 최종 출력 레이어

    def forward(self, src, trg, src_mask, trg_mask):
        # 1. 임베딩 및 Positional Encoding 적용
        # 2. Encoder 실행 (src -> enc_src)
        # 3. Decoder 실행 (trg + enc_src -> output)
        # 4. 최종 출력 (output -> logit)
        pass # 여기에 전체 구현 코드가 들어갑니다.

In [ ]:
import torch
import torch.optim as optim
import numpy as np

# --- (하이퍼파라미터 정의) ---
SRC_VOCAB_SIZE = 100
TRG_VOCAB_SIZE = 100
D_MODEL = 512
N_LAYERS = 3
N_HEAD = 8
D_HID = 2048
MAX_LEN = 50
DROPOUT = 0.1
PAD_IDX = 0
N_EPOCHS = 200

# 🌟 Accuracy 계산 함수 (이전에 PyTorch 내장 트랜스포머에서 사용했던 것과 유사)
def calculate_accuracy(output, target, pad_idx):
    # output: [Seq Len, Batch Size, Vocab Size]
    # target: [Seq Len, Batch Size]
    predicted_tokens = output.argmax(dim=-1)
    mask = (target != pad_idx)
    correct_sum = (predicted_tokens == target).long().masked_select(mask).sum().item()
    total_tokens = mask.sum().item()
    return correct_sum / total_tokens if total_tokens > 0 else 0.0

# 🌟 Copy Task 데이터 생성 함수 (Seq Len=50, 배치 크기 32)
def generate_copy_batch(batch_size, seq_len, vocab_size):
    # [Seq Len, Batch Size] 형태로 생성
    data = torch.randint(1, vocab_size, (seq_len, batch_size))
    # Copy Task이므로 src와 trg가 동일
    return data, data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
import numpy as np
import time

# --- (PositionalEncoding, MultiHeadAttention, TransformerEncoderLayer 클래스가 여기에 정의되어 있어야 합니다.) ---

# 편의를 위해 여기에 간소화된 PositionalEncoding만 재정의합니다.
# 실제 코드는 이전 답변에서 제공된 상세 코드를 사용해야 합니다.
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0) # [1, max_len, d_model]
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x: [Batch Size, Seq Len, d_model]
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [ ]:
# Multi-Head Attention이 포함된 TransformerDecoderLayer가 필요합니다.
# 구현 복잡도를 낮추기 위해 여기서는 PyTorch 내장 Layer를 활용하여 구조만 완성합니다.

class FullTransformer(nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, n_layers, n_head, d_hid, max_len, dropout):
        super().__init__()

        self.d_model = d_model

        # 1. 임베딩 및 Positional Encoding
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(trg_vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len)

        # 2. Encoder Layer Stack
        encoder_layer = nn.TransformerEncoderLayer(d_model, n_head, d_hid, dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, n_layers)

        # 3. Decoder Layer Stack
        decoder_layer = nn.TransformerDecoderLayer(d_model, n_head, d_hid, dropout, batch_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, n_layers)

        # 4. 최종 출력 레이어
        self.fc_out = nn.Linear(d_model, trg_vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.encoder_embedding.weight, -initrange, initrange)
        nn.init.uniform_(self.decoder_embedding.weight, -initrange, initrange)
        nn.init.uniform_(self.fc_out.weight, -initrange, initrange)
        nn.init.zeros_(self.fc_out.bias)

    def forward(self, src, trg, src_mask, trg_mask):
        # src, trg: [Batch Size, Seq Len]

        # 임베딩 + PE: [Batch Size, Seq Len, d_model]
        src_embedded = self.pos_encoder(self.encoder_embedding(src) * math.sqrt(self.d_model))
        trg_embedded = self.pos_encoder(self.decoder_embedding(trg) * math.sqrt(self.d_model))

        # Encoder 출력 (Memory): [Batch Size, Seq Len, d_model]
        enc_src = self.encoder(src_embedded, mask=src_mask)

        # Decoder 출력: [Batch Size, Seq Len, d_model]
        output = self.decoder(trg_embedded, enc_src, tgt_mask=trg_mask, memory_mask=src_mask)

        # 최종 출력: [Batch Size, Seq Len, Trg Vocab Size]
        output = self.fc_out(output)

        return output

In [ ]:
# ----------------------------------------------------
# A. 하이퍼파라미터 및 설정
# ----------------------------------------------------
SRC_VOCAB_SIZE = 12   # 0: PAD, 1: BOS, 2: EOS, 3~11: 실제 토큰 (총 12개)
TRG_VOCAB_SIZE = 12
D_MODEL = 256
N_LAYERS = 3
N_HEAD = 4
D_HID = 512
MAX_LEN = 10
DROPOUT = 0.1
PAD_IDX = 0
N_EPOCHS = 200
BATCH_SIZE = 32
LR = 0.0005

BOS_IDX = 1
EOS_IDX = 2

# ----------------------------------------------------
# B. 유틸리티 함수
# ----------------------------------------------------

def generate_copy_batch(batch_size, seq_len, vocab_size):
    # 실제 토큰 범위 (3 ~ vocab_size-1)
    data = torch.randint(3, vocab_size, (batch_size, seq_len))
    # Copy Task이므로 src와 trg가 동일
    return data, data

def create_mask(seq):
    # Padding Mask (0인 위치는 True)
    pad_mask = (seq == PAD_IDX)
    return pad_mask.unsqueeze(1).unsqueeze(2)

def generate_square_subsequent_mask(size):
    # Look-Ahead Mask (미래 토큰을 보지 못하게 함)
    mask = torch.triu(torch.ones(size, size), diagonal=1).bool()
    return mask

def calculate_accuracy(output, target, pad_idx):
    # output: [Batch Size, Seq Len, Vocab Size]
    # target: [Batch Size, Seq Len]
    predicted_tokens = output.argmax(dim=-1)

    # 패딩이 아닌 위치에 대해서만 마스크 생성
    mask = (target != pad_idx)
    correct_sum = (predicted_tokens == target).long().masked_select(mask).sum().item()
    total_tokens = mask.sum().item()
    return correct_sum / total_tokens if total_tokens > 0 else 0.0

# ----------------------------------------------------
# C. 모델 초기화 및 학습 루프
# ----------------------------------------------------

# 1. 모델 초기화
model = FullTransformer(SRC_VOCAB_SIZE, TRG_VOCAB_SIZE, D_MODEL, N_LAYERS, N_HEAD, D_HID, MAX_LEN, DROPOUT)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
optimizer = optim.Adam(model.parameters(), lr=LR)

# 2. 학습 루프
print("--- Full Transformer 학습 시작 (직접 구현 기반) ---")
model.train()
start_time = time.time()

for epoch in range(1, N_EPOCHS + 1):
    src_data, trg_data = generate_copy_batch(BATCH_SIZE, MAX_LEN, SRC_VOCAB_SIZE)

    # 🌟 타겟 시퀀스 준비: 디코더 입력(trg_input)과 목표(trg_output) 분리
    # Copy Task이므로 src=trg이지만, Seq2Seq는 디코더 입력과 목표가 다름
    trg_input = trg_data[:, :-1]  # 마지막 토큰(EOS) 제외
    trg_output = trg_data[:, 1:]   # 첫 토큰(BOS) 제외

    # 3. 마스크 생성
    src_mask = None # 인코더는 일반적으로 패딩 마스크만 사용하나, Copy Task는 패딩을 생략하여 None으로 둠.

    # 디코더의 룩어헤드 마스크 (trg_input의 길이로 생성)
    trg_mask = generate_square_subsequent_mask(trg_input.size(1))

    # 4. Forward Pass
    output = model(src_data, trg_input, src_mask, trg_mask)

    # output: [Batch Size, Seq Len-1, Trg Vocab Size]
    # trg_output: [Batch Size, Seq Len-1]

    # 5. 손실 계산 (평탄화 후)
    loss = criterion(output.reshape(-1, output.shape[-1]), trg_output.reshape(-1))

    # 6. 정확도 계산
    accuracy = calculate_accuracy(output, trg_output, PAD_IDX)

    # 7. Backward Pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        elapsed = time.time() - start_time
        print(f"Epoch {epoch:3d} | Loss: {loss.item():.4f} | Accuracy: {accuracy:.4f} | Time: {elapsed:.2f}s")
        start_time = time.time() # 시간 초기화

print("--- Full Transformer 학습 완료 ---")

# ----------------------------------------------------
# D. 최종 추론 (Evaluation)
# ----------------------------------------------------

# (복잡성으로 인해 여기서는 간단한 테스트만 포함합니다. 실제 추론은 디코딩 과정이 필요합니다.)
# print("\n--- 최종 평가 결과: 학습 정확도 ---")
# eval_loss = loss.item()
# eval_accuracy = accuracy
# print(f"최종 학습 손실: {eval_loss:.4f}, 최종 학습 정확도: {eval_accuracy:.4f}")

--- Full Transformer 학습 시작 (직접 구현 기반) ---
Epoch  20 | Loss: 1.9937 | Accuracy: 0.2326 | Time: 2.80s
Epoch  40 | Loss: 1.8385 | Accuracy: 0.3056 | Time: 2.27s
Epoch  60 | Loss: 1.4992 | Accuracy: 0.4236 | Time: 2.84s
Epoch  80 | Loss: 1.0344 | Accuracy: 0.6076 | Time: 2.17s
Epoch 100 | Loss: 0.5394 | Accuracy: 0.8229 | Time: 1.89s
Epoch 120 | Loss: 0.2712 | Accuracy: 0.9097 | Time: 1.88s
Epoch 140 | Loss: 0.2387 | Accuracy: 0.9236 | Time: 1.87s
Epoch 160 | Loss: 0.0690 | Accuracy: 0.9757 | Time: 2.38s
Epoch 180 | Loss: 0.1472 | Accuracy: 0.9583 | Time: 1.95s
Epoch 200 | Loss: 0.0812 | Accuracy: 0.9792 | Time: 1.92s
--- Full Transformer 학습 완료 ---
